# Import libraries and dataset

In [218]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# Import Datasets
# this dataset has not dropped any features but encoded
df = pd.read_csv('../data/processed/data_cleaned_v2.csv') 
# import 原始資料方便特徵創造
df_raw = pd.read_csv('/Users/samlai/Documents/Vehicle_insurance_claim_project/data/raw/fraud_oracle.csv')

# 將特徵分組

In [219]:
# 保險人特徵
policyholder_feature = [
    'Sex', 'Age', 'DriverRating_2', 'DriverRating_3', 'DriverRating_4',
    'AgeOfPolicyHolder_18 to 20', 'AgeOfPolicyHolder_21 to 25',
    'AgeOfPolicyHolder_26 to 30', 'AgeOfPolicyHolder_31 to 35', 
    'AgeOfPolicyHolder_36 to 40', 'AgeOfPolicyHolder_41 to 50',
    'AgeOfPolicyHolder_51 to 65', 'AgeOfPolicyHolder_over 65',
    'AddressChange_Claim_2 to 3 years', 'AddressChange_Claim_4 to 8 years',
    'AddressChange_Claim_no change', 'AddressChange_Claim_under 6 months', 'NumberOfCars_2 vehicles',
    'NumberOfCars_3 to 4', 'NumberOfCars_5 to 8', 'NumberOfCars_more than 8',
    'PastNumberOfClaims', 'MaritalStatus_Married', 'MaritalStatus_Single',
    'MaritalStatus_Widow'
]

# 車禍事件特徵 
accident_feature = [
    'AccidentArea', 'Fault', 'PoliceReportFiled', 'NumberOfSuppliments',
    'Days_Policy_Accident_more than 30', 'Days_Policy_Accident_15 to 30',
    'Days_Policy_Accident_8 to 15', 'Days_Policy_Accident_none',
    'Days_Policy_Claim_8 to 15', 'Days_Policy_Claim_more than 30',
    'Days_Policy_Claim_none', 'BasePolicy_Collision', 'BasePolicy_Liability',
    'Deductible_400', 'Deductible_500', 'Deductible_700',
    'PolicyType_Sedan - Collision', 'PolicyType_Sedan - Liability',
    'PolicyType_Sport - Collision', 'PolicyType_Sport - All Perils',
    'PolicyType_Sport - Liability', 'PolicyType_Utility - All Perils',
    'PolicyType_Utility - Collision', 'PolicyType_Utility - Liability', 'WitnessPresent'
]

# 車輛特徵
vehicle_feature = [
    'AgeOfVehicle', 'VehiclePrice_30000 to 39000',
    'VehiclePrice_40000 to 59000', 'VehiclePrice_less than 20000', 'VehiclePrice_60000 to 69000',
    'VehiclePrice_more than 69000', 'Make_Chevrolet', 'Make_Ford',
    'Make_Honda', 'Make_Mazda', 'Make_Pontiac', 'Make_Toyota', 'Make_VW',
    'Make_BMW', 'Make_Dodge', 'Make_Ferrari', 'Make_Jaguar', 'Make_Lexus',
    'Make_Mecedes', 'Make_Mercury', 'Make_Nisson', 'Make_Porche', 'Make_Saab',
    'Make_Saturn', 'VehicleCategory_Sport', 'VehicleCategory_Utility'
]

# 時間特徵
time_feature = [
    'Month_Aug', 'Month_Dec', 'Month_Feb', 'Month_Jan', 'Month_Jul',
    'Month_Jun', 'Month_Mar', 'Month_May', 'Month_Nov', 'Month_Oct',
    'Month_Sep', 'WeekOfMonth_2', 'WeekOfMonth_3', 'WeekOfMonth_4',
    'WeekOfMonth_5', 'DayOfWeek_Monday', 'DayOfWeek_Saturday',
    'DayOfWeek_Sunday', 'DayOfWeek_Thursday', 'DayOfWeek_Tuesday',
    'DayOfWeek_Wednesday', 'Year_1995', 'Year_1996',
    'DayOfWeekClaimed_Friday', 'DayOfWeekClaimed_Monday', 'DayOfWeekClaimed_Saturday',
    'DayOfWeekClaimed_Sunday', 'DayOfWeekClaimed_Thursday', 'DayOfWeekClaimed_Tuesday',
    'DayOfWeekClaimed_Wednesday', 'MonthClaimed_Apr', 'MonthClaimed_Aug',
    'MonthClaimed_Dec', 'MonthClaimed_Feb', 'MonthClaimed_Jan', 'MonthClaimed_Jul',
    'MonthClaimed_Jun', 'MonthClaimed_Mar', 'MonthClaimed_May', 'MonthClaimed_Nov',
    'MonthClaimed_Oct', 'MonthClaimed_Sep', 'WeekOfMonthClaimed_2',
    'WeekOfMonthClaimed_3', 'WeekOfMonthClaimed_4', 'WeekOfMonthClaimed_5'
]

# 其他特徵
other_feature = [
    'AgentType', 'RepNumber_2', 'RepNumber_3', 'RepNumber_4', 'RepNumber_5',
    'RepNumber_6', 'RepNumber_7', 'RepNumber_8', 'RepNumber_9',
    'RepNumber_10', 'RepNumber_11', 'RepNumber_12', 'RepNumber_13',
    'RepNumber_14', 'RepNumber_15', 'RepNumber_16'
]

Target = ['FraudFound_P']


In [220]:
# 檢查是否有特徵未被分組
all_grouped_features = policyholder_feature + accident_feature + vehicle_feature + time_feature + other_feature
all_features = df.columns.tolist()


ungrouped_features = [f for f in all_features if f not in all_grouped_features]
if ungrouped_features:
    print("未分組的特徵:")
    for f in ungrouped_features:
        print(f)
else:
    print("所有特徵都已被分組")

未分組的特徵:
FraudFound_P


In [221]:
df_policyholder = df[policyholder_feature]
df_accident = df[accident_feature]
df_vehicle = df[vehicle_feature]
df_time = df[time_feature]
df_other = df[other_feature]
df_target = df[Target]

# Create a Function to evaluate created features

In [222]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix

# Calculate the accuracy, precision, recall, and F1 score
def get_test_metrics(y_test, y_pred, y_prob):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)

    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')
    print(f'AUC: {auc:.4f}')

    # Calculate and print confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label') 
    plt.show()

In [223]:
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier

def model_test(df_policyholder, df_accident, df_vehicle, df_time, df_other, df_target):
    df_new = pd.concat([df_policyholder, df_accident, df_vehicle, df_time, df_other, df_target], axis=1)
    X = df_new.drop(columns = ['FraudFound_P'])
    y = df_new['FraudFound_P']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

    # use Random Undersampling
    rus = RandomUnderSampler(random_state=42)
    X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train_rus, y_train_rus)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    get_test_metrics(y_test, y_pred, y_prob)

In [224]:
from sklearn.model_selection import StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def cv_evaluation(df_policyholder, df_accident, df_vehicle, df_time, df_other, df_target):
    # 合併所有特徵
    df_new = pd.concat([df_policyholder, df_accident, df_vehicle, df_time, df_other, df_target], axis=1)
    X = df_new.drop(columns=['FraudFound_P'])
    y = df_new['FraudFound_P']
    
    # 定義 5 折交叉驗證
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # 儲存每折的分數
    cv_scores = []
    
    # 進行交叉驗證
    for train_idx, val_idx in cv.split(X, y):
        # 分割數據
        X_train_fold = X.iloc[train_idx]
        y_train_fold = y.iloc[train_idx]
        X_val_fold = X.iloc[val_idx]
        y_val_fold = y.iloc[val_idx]
        
        # 下採樣
        rus = RandomUnderSampler(random_state=42)
        X_train_rus, y_train_rus = rus.fit_resample(X_train_fold, y_train_fold)
        
        # 訓練模型
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train_rus, y_train_rus)
        
        # 預測
        y_val_pred = model.predict(X_val_fold)
        y_val_prob = model.predict_proba(X_val_fold)[:, 1]
        
        # 計算分數
        scores = {
            'accuracy': accuracy_score(y_val_fold, y_val_pred),
            'precision': precision_score(y_val_fold, y_val_pred),
            'recall': recall_score(y_val_fold, y_val_pred),
            'f1': f1_score(y_val_fold, y_val_pred),
            'auc': roc_auc_score(y_val_fold, y_val_prob)
        }
        cv_scores.append(scores)
    
    # 計算平均分數
    print("Average CV scores:")
    for metric in ['accuracy', 'precision', 'recall', 'f1', 'auc']:
        mean_score = np.mean([s[metric] for s in cv_scores])
        print(f"{metric}: {mean_score:.4f}")


# Feature Engineering

## 1. 將Age分群 (結論：已有 'AgeOfPolicyHolder'，刪除Age)

In [225]:
df_policyholder_dropAge = df_policyholder.drop(columns = 'Age')
cv_evaluation(df_policyholder_dropAge, df_accident, df_vehicle, df_time, df_other, df_target)

Average CV scores:
accuracy: 0.6553
precision: 0.1369
recall: 0.8960
f1: 0.2375
auc: 0.8157


In [226]:
# 移除Age後，成效有些為提升，因此將Age移除
df_policyholder = df_policyholder.drop(columns = 'Age')

## 2.創造高風險車子品牌標籤特徵 (結論：保留新建的高風險特徵，移除所有Brand Make特徵)

In [227]:
# Create HighRisk_brand feature based on specific makes
# Mecedes, BMW, Saab, Saturn, BMW, Ford, Mercury 這些為高風險品牌
# Accurca 是One-hot Encoding後的基準類別，其他 Make features若皆為 0 就是Accurca
high_risk_makes = ['Make_Mecedes', 'Make_Saab', 'Make_Saturn', 'Make_BMW', 'Make_Ford', 'Make_Mercury']

# Get all make columns
all_makes = ['Make_Chevrolet', 'Make_Ford', 'Make_Honda', 'Make_Mazda', 'Make_Pontiac', 
            'Make_Toyota', 'Make_VW', 'Make_BMW', 'Make_Dodge', 'Make_Ferrari', 
            'Make_Jaguar', 'Make_Lexus', 'Make_Mecedes', 'Make_Mercury', 'Make_Nisson',
            'Make_Porche', 'Make_Saab', 'Make_Saturn']

# Create HighRisk_brand feature
df_vehicle['HighRisk_brand'] = ((df_vehicle[high_risk_makes].any(axis=1)) | 
                               (df_vehicle[all_makes].sum(axis=1) == 0)).astype(int)


/var/folders/th/klmxbc1d2x58bny86xqxsjqc0000gn/T/ipykernel_52308/1558674046.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vehicle['HighRisk_brand'] = ((df_vehicle[high_risk_makes].any(axis=1)) |


In [228]:
cv_evaluation(df_policyholder, df_accident, df_vehicle, df_time, df_other, df_target)

Average CV scores:
accuracy: 0.6548
precision: 0.1370
recall: 0.8981
f1: 0.2377
auc: 0.8171


In [229]:
# 試著把Make相關features都刪掉，因為有HighRisk_brand了
df_vehicle_withoutMake = df_vehicle.drop(columns = all_makes)
cv_evaluation(df_policyholder, df_accident, df_vehicle_withoutMake, df_time, df_other, df_target)

Average CV scores:
accuracy: 0.6543
precision: 0.1353
recall: 0.8851
f1: 0.2347
auc: 0.8178


In [230]:
# 將18個Brand Make相關feature都刪除後，recall跟F1 score反而有些為提升，表示一個HighRisk_brand就可以提供足夠資訊
# 更新df_vehicle，移除 Brand Make 相關欄位
df_vehicle = df_vehicle.drop(columns = all_makes)

## 3.移除PolicyType 或BasePolicy

In [231]:
# PolicyType Column為Based Policy與Vehicle Category的組合，移除PolictType測試
# Get all PolicyType columns
policy_type_cols = [col for col in df_accident.columns if col.startswith('PolicyType')]

# Remove PolicyType columns from df_accident
df_accident_dropPolicyType = df_accident.drop(columns=policy_type_cols)

cv_evaluation(df_policyholder, df_accident_dropPolicyType, df_vehicle, df_time, df_other, df_target)

Average CV scores:
accuracy: 0.6569
precision: 0.1354
recall: 0.8776
f1: 0.2346
auc: 0.8172


In [232]:
# 移除Based Policy與Vehicle Category
# Get all Based Policy columns
BasePolicy_cols = [col for col in df_accident.columns if col.startswith('BasePolicy')]
df_accident_dropBasePolicy = df_accident.drop(columns=BasePolicy_cols)

VehicleCategory_col = [col for col in df_vehicle.columns if col.startswith('VehicleCategory')]
df_vehicle_dropCategory = df_vehicle.drop(columns=VehicleCategory_col)

cv_evaluation(df_policyholder, df_accident_dropBasePolicy, df_vehicle_dropCategory, df_time, df_other, df_target)

Average CV scores:
accuracy: 0.6690
precision: 0.1398
recall: 0.8765
f1: 0.2410
auc: 0.8231


In [233]:
# 移除Based Policy與Vehicle Category在交叉驗證瞟線上較好
# 更新df_accident與df_vehicle
df_accident = df_accident.drop(columns=BasePolicy_cols)
df_vehicle = df_vehicle.drop(columns=VehicleCategory_col)

## 4.'Days_Policy_Accident' and 'Days_Policy_Claim' Columns

In [234]:
# Create Days_Policy_Accident_morethan30 feature
days_policy_accident_col = [col for col in df_accident.columns if col.startswith('Days_Policy_Accident')]
df_accident = df_accident.drop(columns=days_policy_accident_col)
df_accident['Days_Policy_Accident_morethan30'] = (df_raw['Days_Policy_Accident'] == 'more than 30').astype(int)


In [235]:
# Create Days_Policy_Claim_morethan30 feature
days_policy_claim_col = [col for col in df_accident.columns if col.startswith('Days_Policy_Claim')]
df_accident = df_accident.drop(columns=days_policy_claim_col)
df_accident['Days_Policy_Claim_morethan30'] = (df_raw['Days_Policy_Claim'] == 'more than 30').astype(int)

In [236]:
cv_evaluation(df_policyholder, df_accident, df_vehicle, df_time, df_other, df_target)

Average CV scores:
accuracy: 0.6682
precision: 0.1383
recall: 0.8678
f1: 0.2386
auc: 0.8200


In [237]:
# 因為兩個特徵非常相似，試著刪掉其中一個
df_accident_dropAccident = df_accident.drop(columns = 'Days_Policy_Accident_morethan30')
cv_evaluation(df_policyholder, df_accident_dropAccident, df_vehicle, df_time, df_other, df_target)
print('-'*100)
df_accident_dropClaim = df_accident.drop(columns = 'Days_Policy_Claim_morethan30')
cv_evaluation(df_policyholder, df_accident_dropClaim, df_vehicle, df_time, df_other, df_target)

Average CV scores:
accuracy: 0.6617
precision: 0.1342
recall: 0.8516
f1: 0.2318
auc: 0.8221
----------------------------------------------------------------------------------------------------
Average CV scores:
accuracy: 0.6620
precision: 0.1360
recall: 0.8668
f1: 0.2350
auc: 0.8180


In [238]:
# Calculate correlation between Days_Policy_Accident_morethan30 and Days_Policy_Claim_morethan30
correlation = df_accident['Days_Policy_Accident_morethan30'].corr(df_accident['Days_Policy_Claim_morethan30'])
print(f"Correlation between 2 variables: {correlation:.4f}")

# drop 'Days_Policy_Claim_morethan30'
df_accident = df_accident.drop(columns = 'Days_Policy_Claim_morethan30')

Correlation between 2 variables: 0.6347


## 5. 合併AddressChange_Claim, Number of Cars中的少數別

In [239]:
# 將AddressChange_Claim中的'under 6 months', '1 year', '2 to 3 years'合併為 'under 3 years'
# 但由於只剩 'under 3 years', 'no change', '4 to 8 years'三個選項，
# 因此透過'no change', '4 to 8 years'就可以推斷出'under 3 years'欄位的值，
# 不需要創造 'AddressChange_Claim_under 3 years'欄位
df_policyholder = df_policyholder.drop(columns = ['AddressChange_Claim_2 to 3 years', 'AddressChange_Claim_under 6 months'])

In [240]:
df_policyholder['More than 1 car'] = (df_raw['NumberOfCars'] != '1 vehicle').astype(int)
NumberOfCars_col = [col for col in df_policyholder.columns if col.startswith('NumberOfCars')]
df_policyholder = df_policyholder.drop(columns=NumberOfCars_col)

## 6.